In [2]:
from model import *
import pandas as pd

In [8]:
abalone = pd.read_csv('synthetic/abalone/real.csv')

In [9]:
test = pd.read_csv('synthetic/abalone/test.csv')

In [35]:
ttvae = TTVAE(epochs=1)

In [14]:
abalone.columns

Index(['sex', 'length', 'diameter', 'height', 'wholeweight', 'shuckedweight',
       'visceraweight', 'shellweight', 'rings'],
      dtype='object')

In [16]:
discrete_columns = abalone.columns[abalone.dtypes=='object']

In [17]:
discrete_columns

Index(['sex'], dtype='object')

In [19]:
ttvae.fit(abalone, discrete_columns, 'ttvae/ckpt')

/home/binhducvu/anaconda3/lib/python3.9/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
/home/binhducvu/anaconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


84 32 128 8 1028 0.1


Epoch 134/300: 100%|████████████████████████████████████████████████████████████| 7/7 [00:00<00:00,  9.97it/s, Loss=-45]


Epoch 268/300: 100%|██████████████████████████████████████████████████████████| 7/7 [00:00<00:00,  9.18it/s, Loss=-58.7]


Epoch 300/300: 100%|████████████████████████████████████████████████████████████| 7/7 [00:00<00:00,  8.11it/s, Loss=-64]


In [24]:
transformed = ttvae.transformer.transform(test)
transformed = torch.from_numpy(transformed.astype('float32')).to('cuda:0')

In [31]:
mu, std, logvar, enc_embed = ttvae.encoder(transformed)
synthetic_embeddings=mu
noise = torch.Tensor(synthetic_embeddings).to(ttvae._device)
ttvae.decoder.eval()
with torch.no_grad():
    fake, sigmas = ttvae.decoder(noise,enc_embed)
    fake = torch.tanh(fake).cpu().detach().numpy()
final = ttvae.transformer.inverse_transform(fake)

In [32]:
final

,sex,length,diameter,height,wholeweight,shuckedweight,visceraweight,shellweight,rings
0,F,0.582236,0.458723,0.164730,0.979701,0.345323,0.252814,0.307930,12
1,I,0.407512,0.304142,0.092103,0.291460,0.140331,0.057360,0.071587,8
2,M,0.281496,0.219529,0.074950,0.110056,0.039673,0.019856,0.030733,5
3,M,0.613830,0.462636,0.170174,1.058509,0.477705,0.274402,0.267761,10
4,F,0.470045,0.355667,0.115330,0.490489,0.192078,0.130016,0.151151,8
...,...,...,...,...,...,...,...,...,...
831,F,0.391565,0.298649,0.096326,0.232699,0.077161,0.051461,0.076708,10
832,F,0.478929,0.326496,0.137793,0.577165,0.212173,0.112431,0.207373,15
833,I,0.555228,0.426127,0.144914,0.804024,0.345306,0.179452,0.221822,9
834,M,0.465924,0.363853,0.080361,0.488604,0.188113,0.125006,0.151681,11


In [33]:
test

,sex,length,diameter,height,wholeweight,shuckedweight,visceraweight,shellweight,rings
0,F,0.585,0.455,0.165,0.9980,0.3450,0.2495,0.315,12
1,I,0.410,0.300,0.090,0.2800,0.1410,0.0575,0.075,8
2,M,0.285,0.215,0.075,0.1060,0.0415,0.0230,0.035,5
3,M,0.615,0.460,0.170,1.0565,0.4815,0.2720,0.270,10
4,F,0.470,0.350,0.115,0.4870,0.1955,0.1270,0.155,8
...,...,...,...,...,...,...,...,...,...
831,F,0.395,0.295,0.095,0.2245,0.0780,0.0540,0.080,10
832,F,0.465,0.390,0.140,0.5555,0.2130,0.1075,0.215,15
833,I,0.555,0.425,0.145,0.7905,0.3485,0.1765,0.225,9
834,M,0.465,0.360,0.080,0.4880,0.1910,0.1250,0.155,11
